### 0. Load Library & Data

In [134]:
# Libraries
import os
import re
import jsonlines
import pandas as pd
import ast
import ast
import copy
import warnings
warnings.filterwarnings('ignore')

In [136]:
# Load Original LitBank Json Data
litbank_jsons=[]
with jsonlines.open("data/LitBank/train.english.jsonlines") as read_file:
    for line in read_file.iter():
        litbank_jsons.append(line)

# Load IAA LitBank Data
# 폴더 내 파일 정렬
litbank_dfs_folder = os.listdir("data/NewLitBank_IAA")
litbank_dfs_folder = sorted(litbank_dfs_folder, key=lambda x: int(re.search(r'\d+', x).group()))

# 케이스별 처리 함수
def safe_literal_eval(val):
    try:
        return ast.literal_eval(val)
    except (ValueError, SyntaxError):
        return val  

def load_and_process_litbank(case_value):
    dfs = []
    for fname in litbank_dfs_folder:
        df = pd.read_csv(f"data/NewLitBank_IAA/{fname}").drop(
            columns=['Unnamed: 0', 'labels', 'inference_offsets'], errors='ignore'
        )
        df = df.loc[df['cases'] == case_value].copy()
        for col in df.columns:
            df[col] = df[col].apply(safe_literal_eval)
        dfs.append(df)
    return dfs

best_litbank_dfs = load_and_process_litbank(0)
worst_litbank_dfs = load_and_process_litbank(1)
weird_litbank_dfs = load_and_process_litbank(2)
litbank_dfs = best_litbank_dfs

### 1. 증강된 coref의 offset 변화량을 json cluster에 반영 

In [137]:
## 1. 증강된 만큼 clusters를 늘려준다.

aug_litbank_jsons = []

for i in range(len(litbank_jsons)):
    item = litbank_jsons[i]
    num_rows = len(litbank_dfs[i])  # 해당 df의 행 개수

    for j in range(num_rows):
        new_item = {
            "doc_key": f"{item['doc_key']}_{j}",
            "sentences": item["sentences"],
            "clusters": item["clusters"]
        }
        aug_litbank_jsons.append(new_item)

print(f"증강된 json 개수: {len(aug_litbank_jsons)}")

증강된 json 개수: 186


In [ ]:
## 2. update_coref의 offset 변화량을 coref에 반영 -> reverse_coref 생성

def apply_reverse_coref(litbank_dfs):
    """
    각 DataFrame의 'coref', 'adjusted_offsets', 'update_coref'를 기반으로
    'reverse_coref'를 계산하여 새로운 컬럼으로 추가한다.

    Parameters:
        litbank_dfs (list): 여러 개의 pandas DataFrame을 담고 있는 리스트

    Returns:
        list: 'reverse_coref' 컬럼이 추가된 DataFrame 리스트
    """
    updated_dfs = []

    for litbank_df in litbank_dfs:
        # coref 정렬
        litbank_df['coref'] = litbank_df['coref'].apply(lambda cluster: sorted(cluster, key=lambda x: x[0]))

        reverse_coref = []
        for i in range(len(litbank_df)):
            coref = litbank_df.iloc[i]['coref']
            adjusted_offsets = litbank_df.iloc[i]['adjusted_offsets']
            update_coref = litbank_df.iloc[i]['update_coref']

            reverse = []
            for j in range(len(update_coref)):
                start_delta = update_coref[j][0] - adjusted_offsets[j][0] # start offset 증가량
                end_delta = update_coref[j][1] - adjusted_offsets[j][1] # end offset 증가량
                new_start = coref[j][0] + start_delta # start offset 증가량 반영
                new_end = coref[j][1] + end_delta # end offset 증가량 반영
                reverse.append([new_start, new_end]) # 증가량 반영해서 coref offset 변형한 offset

            reverse_coref.append(reverse)

        litbank_df['reverse_coref'] = reverse_coref
        updated_dfs.append(litbank_df)
    return updated_dfs

In [139]:
# 전체 데이터프레임에 적용 후 하나의 데이터프레임으로 합치기
updated_litbank_dfs = apply_reverse_coref(litbank_dfs)
total_litbank_dfs = pd.concat(updated_litbank_dfs, ignore_index=True)
print('전체 증강된 케이스 개수 : ', len(total_litbank_dfs))
print('첫번째 증강 케이스 예시')
total_litbank_dfs[:3]

전체 증강된 케이스 개수 :  186
첫번째 증강 케이스 예시


,coref,extracted_sentence,adjusted_offsets,text,update_text,update_sentence,update_coref,cases,reverse_coref
0,"[[43, 53], [144, 145], [605, 606]]","[There, lived, the, colliers, who, worked, in,...","[[2, 12], [30, 31], [39, 40]]","[[the, colliers, who, worked, in, the, little,...","[[the, colliers, who, worked, in, the, little,...","[There, lived, the, colliers, who, worked, in,...","[[2, 12], [30, 32], [40, 42]]",0,"[[43, 53], [144, 146], [606, 608]]"
1,"[[209, 209], [393, 393]]","[The, coal, and, iron, field, of, Nottinghamsh...","[[8, 8], [92, 92]]","[[Derbyshire], [Derbyshire]]","[[Derbyshire], [picturesque, Derbyshire]]","[The, coal, and, iron, field, of, Nottinghamsh...","[[8, 8], [92, 93]]",0,"[[209, 209], [393, 394]]"
2,"[[296, 296], [382, 382]]","[From, Nuttall, ,, high, up, on, the, sandston...","[[69, 69], [125, 125]]","[[Selby], [Selby]]","[[Selby], [the, nearby, Selby]]","[From, Nuttall, ,, high, up, on, the, sandston...","[[69, 69], [125, 127]]",0,"[[296, 296], [382, 384]]"


In [140]:
## 3. reverse_coref를 기반으로 clusters를 업데이트, 이후 클러스터에 변화량 반영

for idx, row in enumerate(total_litbank_dfs.itertuples()):
    # 대상 clusters 리스트
    clusters = copy.deepcopy(aug_litbank_jsons[idx]['clusters'])
    # 클러스터 정렬렬
    [cluster.sort(key=lambda x: x[0]) for cluster in clusters]

    # DataFrame에서 coref, reverse_coref 가져오기
    coref = row.coref
    reverse_coref = row.reverse_coref

    # 문자열이면 리스트로 변환
    if isinstance(coref, str):
        coref = ast.literal_eval(coref)
    if isinstance(reverse_coref, str):
        reverse_coref = ast.literal_eval(reverse_coref)

    # 변화량 추적용 리스트
    change_points = []
    for j in range(len(coref)):
        original = coref[j]
        updated = reverse_coref[j]

        original_length = original[1] - original[0]
        updated_length = updated[1] - updated[0]
        shift = updated_length - original_length

        if shift != 0:
            change_points.append((original[0], shift))

    # coref와 일치하는 cluster를 찾아 교체
    changed = False
    changed_cluster_idx = None  # 몇 번째 cluster가 바뀌었는지 저장
    for c_idx, cluster in enumerate(clusters):
        if cluster == coref:
            clusters[c_idx] = reverse_coref
            changed = True
            changed_cluster_idx = c_idx
            break

    # shift 기준으로 이후 mention 위치 조정
    for base_offset, shift in change_points:
        for cluster in clusters:
            for mention in cluster:
                if mention[0] > base_offset:
                    mention[0] += shift
                    mention[1] += shift

    # 반영
    aug_litbank_jsons[idx]['clusters'] = clusters

    # 변경 로그 출력
    if changed:
        print(f"[{idx}] cluster 변경됨 → {changed_cluster_idx}번째 cluster가 교체됨")
        print(f"변화 기준점 (start offset 기준): {change_points}")
        print("이전 cluster:", coref)
        print("변경된 cluster:", reverse_coref)
        print("-" * 60)


[0] cluster 변경됨 → 7번째 cluster가 교체됨
변화 기준점 (start offset 기준): [(144, 1), (605, 1)]
이전 cluster: [[43, 53], [144, 145], [605, 606]]
변경된 cluster: [[43, 53], [144, 146], [608, 610]]
------------------------------------------------------------
[1] cluster 변경됨 → 29번째 cluster가 교체됨
변화 기준점 (start offset 기준): [(393, 1)]
이전 cluster: [[209, 209], [393, 393]]
변경된 cluster: [[209, 209], [393, 394]]
------------------------------------------------------------
[2] cluster 변경됨 → 36번째 cluster가 교체됨
변화 기준점 (start offset 기준): [(382, 2)]
이전 cluster: [[296, 296], [382, 382]]
변경된 cluster: [[296, 296], [382, 384]]
------------------------------------------------------------
[3] cluster 변경됨 → 7번째 cluster가 교체됨
변화 기준점 (start offset 기준): [(1352, 2)]
이전 cluster: [[71, 71], [1352, 1352]]
변경된 cluster: [[71, 71], [1352, 1354]]
------------------------------------------------------------
[4] cluster 변경됨 → 27번째 cluster가 교체됨
변화 기준점 (start offset 기준): [(1849, 2)]
이전 cluster: [[413, 413], [1849, 1849]]
변경된 cluster: [[413, 41

### 2. 증강된 update sentence를 json sentences에 반영 

In [141]:
extracted = total_litbank_dfs['extracted_sentence'].iloc[0]
update = total_litbank_dfs['update_sentence'].iloc[0]

text = total_litbank_dfs['text'].iloc[0]
update_text = total_litbank_dfs['update_text'].iloc[0]

sentence = aug_litbank_jsons[0]['sentences'][5]
sentences = aug_litbank_jsons[0]['sentences']

update_sentences = copy.deepcopy(sentences)
for t, u in zip(text, update_text): 
    for s_idx, sent in enumerate(update_sentences):
        for i in range(len(sent) - len(t) + 1):
            if sent[i:i + len(t)] == t:
                # 일치하는 부분을 update_text로 교체
                update_sentences[s_idx] = sent[:i] + u + sent[i + len(t):]
                break  # 한 번만 바꾸고 그 문장은 넘어감

print('text : ', text)
print('update_text : ', update_text)
print('sentences : ', sentences[5])
print('update_sentence : ', update_sentences[5])

text :  [['the', 'colliers', 'who', 'worked', 'in', 'the', 'little', 'gin-pits', 'two', 'fields', 'away'], ['the', 'colliers'], ['these', 'coal-miners']]
update_text :  [['the', 'colliers', 'who', 'worked', 'in', 'the', 'little', 'gin-pits', 'two', 'fields', 'away'], ['the', 'hardworking', 'colliers'], ['these', 'dedicated', 'coal-miners']]
sentences :  ['And', 'the', 'cottages', 'of', 'these', 'coal-miners', ',', 'in', 'blocks', 'and', 'pairs', 'here', 'and', 'there', ',', 'together', 'with', 'odd', 'farms', 'and', 'homes', 'of', 'the', 'stockingers', ',', 'straying', 'over', 'the', 'parish', ',', 'formed', 'the', 'village', 'of', 'Bestwood', '.']
update_sentence :  ['And', 'the', 'cottages', 'of', 'these', 'dedicated', 'coal-miners', ',', 'in', 'blocks', 'and', 'pairs', 'here', 'and', 'there', ',', 'together', 'with', 'odd', 'farms', 'and', 'homes', 'of', 'the', 'stockingers', ',', 'straying', 'over', 'the', 'parish', ',', 'formed', 'the', 'village', 'of', 'Bestwood', '.']


In [142]:
## 4. clusters와 text 비교 후, 일치하는 부분을 update_text로 교체
for idx, row in enumerate(total_litbank_dfs.itertuples()):
    text = row.text 
    update_text = row.update_text

    if isinstance(text, str):
        text = ast.literal_eval(text)
    if isinstance(update_text, str):
        update_text = ast.literal_eval(update_text)

    sentences = aug_litbank_jsons[idx]['sentences']
    update_sentences = copy.deepcopy(sentences)

    for t, u in zip(text, update_text):
        for s_idx, sent in enumerate(update_sentences):
            i = 0
            while i <= len(sent) - len(t):
                # 부분 일치 찾기
                if sent[i:i + len(t)] == t:
                    # 실제로 바뀌는 경우만 교체
                    if t != u:
                        before = update_sentences[s_idx]
                        sent = sent[:i] + u + sent[i + len(t):]
                        update_sentences[s_idx] = sent  # 수정된 문장 반영

                        print(f"[{idx}] {s_idx}번째 문장에서 부분 교체 발생:")
                        print("▶ 교체 전:", before)
                        print("▶ 교체 후:", sent)
                        print("-" * 60)
                        # i를 이동해 중복 방지
                        i += len(u)
                    else:
                        i += 1  # 일치하지만 변경사항 없음
                else:
                    i += 1  # 일치하지 않음 → 다음 위치 검사

    # 반영
    aug_litbank_jsons[idx]['sentences'] = update_sentences


[0] 2번째 문장에서 부분 교체 발생:
▶ 교체 전: ['There', 'lived', 'the', 'colliers', 'who', 'worked', 'in', 'the', 'little', 'gin-pits', 'two', 'fields', 'away', '.']
▶ 교체 후: ['There', 'lived', 'the', 'hardworking', 'colliers', 'who', 'worked', 'in', 'the', 'little', 'gin-pits', 'two', 'fields', 'away', '.']
------------------------------------------------------------
[0] 18번째 문장에서 부분 교체 발생:
▶ 교체 전: ['But', 'that', 'was', 'outside', ';', 'that', 'was', 'the', 'view', 'on', 'to', 'the', 'uninhabited', 'parlours', 'of', 'all', 'the', 'colliers', "'", 'wives', '.']
▶ 교체 후: ['But', 'that', 'was', 'outside', ';', 'that', 'was', 'the', 'view', 'on', 'to', 'the', 'uninhabited', 'parlours', 'of', 'all', 'the', 'hardworking', 'colliers', "'", 'wives', '.']
------------------------------------------------------------
[0] 5번째 문장에서 부분 교체 발생:
▶ 교체 전: ['And', 'the', 'cottages', 'of', 'these', 'coal-miners', ',', 'in', 'blocks', 'and', 'pairs', 'here', 'and', 'there', ',', 'together', 'with', 'odd', 'farms', 'and', 

In [ ]:
with jsonlines.open("data/litbank_augment_iaa.jsonlines", mode='w') as writer:
    for item in aug_litbank_jsons:
        writer.write(item)

### 3. DataFram -> Jsonlines로 잘 변환되었는지 확인

In [144]:
# Litbank Original Data
litbank_jsons=[]
with jsonlines.open("data/LitBank/train.english.jsonlines") as read_file:
    for line in read_file.iter():
        litbank_jsons.append(line)
litbank_df = pd.DataFrame(litbank_jsons)[['doc_key', 'sentences', 'clusters']]

# Litbank IAA Data
litbank_iaa_jsons=[]
with jsonlines.open("data/litbank_augment_iaa.jsonlines") as read_file:
    for line in read_file.iter():
        litbank_iaa_jsons.append(line)
litbank_iaa_df = pd.DataFrame(litbank_iaa_jsons)[['doc_key', 'sentences', 'clusters']]

In [145]:
extracted = total_litbank_dfs['extracted_sentence'].iloc[0]
update = total_litbank_dfs['update_sentence'].iloc[0]

text = total_litbank_dfs['text'].iloc[0]
update_text = total_litbank_dfs['update_text'].iloc[0]

litbank_json_sentences = litbank_jsons[0]['sentences']
litbank_iaa_json_sentesnces = litbank_iaa_jsons[0]['sentences']
litbank_json_clusters = litbank_jsons[0]['clusters']
litbank_iaa_json_clusters = litbank_iaa_jsons[0]['clusters']

print('text : ', text[2])
print('update_text : ', update_text[2])
print('litbank_json_sentences : ', litbank_json_sentences[5]) # these coal-miners
print('litbank_iaa_json_sentences : ', litbank_iaa_json_sentesnces[5]) # these dedicated coal-miners

text :  ['these', 'coal-miners']
update_text :  ['these', 'dedicated', 'coal-miners']
litbank_json_sentences :  ['And', 'the', 'cottages', 'of', 'these', 'coal-miners', ',', 'in', 'blocks', 'and', 'pairs', 'here', 'and', 'there', ',', 'together', 'with', 'odd', 'farms', 'and', 'homes', 'of', 'the', 'stockingers', ',', 'straying', 'over', 'the', 'parish', ',', 'formed', 'the', 'village', 'of', 'Bestwood', '.']
litbank_iaa_json_sentences :  ['And', 'the', 'cottages', 'of', 'these', 'dedicated', 'coal-miners', ',', 'in', 'blocks', 'and', 'pairs', 'here', 'and', 'there', ',', 'together', 'with', 'odd', 'farms', 'and', 'homes', 'of', 'the', 'stockingers', ',', 'straying', 'over', 'the', 'parish', ',', 'formed', 'the', 'village', 'of', 'Bestwood', '.']


### 4. Json파일 Split

In [131]:
# 문서 구조 평탄화
def preprocess_litbank_doc(doc):
    tokens = []
    sentence_spans = []
    idx = 0
    for sentence in doc["sentences"]: # 문서의 문장들을 하나씩 꺼내면서
        tokens.extend(sentence) # 전체 토큰 리스트에 문장 토큰들을 이어 붙임
        start_idx = idx # 현재 문장의 시작 토큰 인덱스 저장
        end_idx = idx + len(sentence) - 1 # 현재 문장의 마지막 토큰 인덱스 계산
        sentence_spans.append((start_idx, end_idx)) # 문장의 토큰 범위(시작, 끝)를 저장
        idx = end_idx + 1 # 다음 문장을 위해 인덱스를 갱신

    return {
        "doc_key": doc["doc_key"],  # 문서 고유 ID
        "sentences": doc["sentences"],  # 문장의 원본 정보
        "clusters": doc["clusters"],  # coreference span 정보
        "tokens": tokens,  # 전체 문장을 하나로 이어붙인 토큰 리스트
        "sentence_spans": sentence_spans  # 각 문장이 문서 전체에서 차지하는 범위
    }

# 문서 나누기 (마지막 문서만 3등분)
def split_document(doc, is_last_doc=False):
    tokens = doc["tokens"] # 전체 토큰 리스트
    sentence_spans = doc["sentence_spans"] # 문장 별 토큰 범위 리스트
    num_tokens = len(tokens)  # 전체 토큰 수

    split_count = 3 if is_last_doc else 2 # 마지막 문서면 3등분, 아니면 2등분
    split_points = [i * num_tokens // split_count for i in range(1, split_count)] # 나누는 기준점 리스트
    # 마지막 문서 아닐 때 : num_tokens  2006 -> split_points = [1003] # 1003번째에서 나눔
    # 마지막 문서 일 때 : num_tokens = 2006 -> split_points = [668, 1337] # 668번째와 1337번째에서 나눔

    split_idxs = [] # 실제 문장을 자를 인덱스 리스트 (문장 단위로 나눌 것)
    for sp in split_points: # 각 분할 기준에 대해서(마지막 문장 아니면 그냥 중간값인 split_points)
        # ex: sp = 1003
        for i, (start, end) in enumerate(sentence_spans): # 문장 범위를 돌면서
            # ex: i = 0, (start,end) = (0,21) -> 첫 번째 문장의 길이가 22이므로 시작 인덱스는 0, 끝 인덱스는 21
            if end >= sp: # 문장 분할 기준을 포함하는 첫 문장(spilt_tokens가 8이면 8번째 문장)
                #  end = ? > sp = 1003 되는 순
                split_idxs.append(i + 1) # 그 다음 문장부터 자르기 위해 +1(9번째 문장)
                # ex: split_idxs = [38] 
                break

    # 구간 생성 : [처음 시작 문장 인덱스, 문장 자르는 문장 인덱스, 마지막 문장 인덱스]
    # [0] + [38] + [106] -> [0, 38, 106] -> 0번째 문장부터 38번째 문장까지, 38번째 문장부터 106번째 문장까지
    split_idxs = [0] + split_idxs + [len(doc["sentences"])] 

    split_docs = [] # 실제 문장을 자를 인덱스 리스트 (문장 단위로 나눌 것)
    for i in range(split_count): # 나눌 문서 수만큼 반복(2번, 마지막 문장만 3번)
        sent_start, sent_end = split_idxs[i], split_idxs[i + 1] # 문장 인덱스 범위 추출
        # 처음 반복 : 0, 38
        # 두 번째 반복 : 38, 106
        token_offset = sentence_spans[sent_start][0] if sent_start < len(sentence_spans) else 0  # 전체 문서에서 현재 부분 문서 시작하는 첫 토큰 위치
        # ex: token_offset = 0
        # ex: token_offset = 1016
        new_sentences = doc["sentences"][sent_start:sent_end] # 해당 범위의 문장들을 잘라냄
        new_token_count = sum(len(s) for s in new_sentences) # 해당 범위의 전체 토큰 수 계산
        # ex: 1번째 token 수=1016, 2번째 token 수=990 -> 1016 + 990 = 2006(전체 토큰 수)
        
        new_clusters = [] # 새 문서에서 사용할 클러스터 리스트
        for cluster in doc["clusters"]: # 원래 문서의 각 클러스터를 확인하면서
            new_cluster = [] # 클러스터 내에서 새로운 span을 저장
            for span in cluster: # 각 클러스터의 span
                if token_offset <= span[0] <= span[1] < token_offset + new_token_count:
                    # 이 span이 현재 분할 범위 안에 **완전히** 포함된다면
                    new_cluster.append([span[0] - token_offset, span[1] - token_offset])
                    # 새로운 문서 기준으로 인덱스를 보정해서 저장
            if new_cluster: # 유효한 span만 모은 클러스터를 저장
                new_clusters.append(new_cluster)

        new_doc = {
            "doc_key": f"{doc['doc_key']}_{i}",
            "sentences": new_sentences,
            "clusters": new_clusters,
        }
        split_docs.append(new_doc)

    return split_docs

In [132]:
# Load LitBank IAA Json Data
litbank_iaa_jsons=[]
with jsonlines.open("data/litbank_augment_iaa.jsonlines") as read_file:
    for line in read_file.iter():
        litbank_iaa_jsons.append(line)
        
# 전체 파이프라인 실행
process_litbank_iaa_jsons = []
for doc in litbank_iaa_jsons: # 하나의 문서에 대해(doc_key 한 개 해당)
    processed = preprocess_litbank_doc(doc)
    if processed:
        process_litbank_iaa_jsons.append(processed)
        
litbank_iaa_splitted_jsons = []
for doc in process_litbank_iaa_jsons:
    litbank_iaa_splitted_jsons.extend(split_document(doc))

print(f"✅ 분할 문서 : IAA LitBank Data")
print(f"✅ 원래 문서 수: {len(litbank_iaa_jsons)}")
print(f"✅ 분할된 문서 수: {len(litbank_iaa_splitted_jsons)}")  

with jsonlines.open("data/splitted_litbank_augment_iaa.jsonlines", mode='w') as writer:
    for doc in litbank_iaa_splitted_jsons:
        writer.write(doc)

✅ 분할 문서 : IAA LitBank Data
✅ 원래 문서 수: 186
✅ 분할된 문서 수: 372


In [133]:
# Load LitBank Json Data
litbank_jsons=[]
with jsonlines.open("data/LitBank/train.english.jsonlines") as read_file:
    for line in read_file.iter():
        litbank_jsons.append(line)

# 전체 파이프라인 실행
process_litbank_jsons = []
for doc in litbank_jsons:
    processed = preprocess_litbank_doc(doc)
    if processed:
        process_litbank_jsons.append(processed)

# 문서 분할 (마지막 문서만 3등분)
litbank_splitted_jsons = []
for idx, doc in enumerate(process_litbank_jsons):
    is_last = (idx == len(process_litbank_jsons) - 1)
    litbank_splitted_jsons.extend(split_document(doc, is_last_doc=is_last))

print(f"✅ 분할 문서 : LitBank Data")
print(f"✅ 원래 문서 수: {len(litbank_jsons)}")
print(f"✅ 분할된 문서 수: {len(litbank_splitted_jsons)}")

# 결과 저장
with jsonlines.open("data/new_train-splitted.english.jsonlines", mode='w') as writer:
    for doc in litbank_splitted_jsons:
        writer.write(doc)

✅ 분할 문서 : LitBank Data
✅ 원래 문서 수: 80
✅ 분할된 문서 수: 161


- Checking

In [87]:
# Load LitBank Json Data
litbank_jsons=[]
with jsonlines.open("data/LitBank/train.english.jsonlines") as read_file:
    for line in read_file.iter():
        litbank_jsons.append(line)
        
def preprocess_litbank_doc(doc):
    tokens = []
    sentence_spans = []
    idx = 0
    for sentence in doc["sentences"]: # 문서의 문장들을 하나씩 꺼내면서
        tokens.extend(sentence) # 전체 토큰 리스트에 문장 토큰들을 이어 붙임
        start_idx = idx # 현재 문장의 시작 토큰 인덱스 저장
        end_idx = idx + len(sentence) - 1 # 현재 문장의 마지막 토큰 인덱스 계산
        sentence_spans.append((start_idx, end_idx)) # 문장의 토큰 범위(시작, 끝)를 저장
        idx = end_idx + 1 # 다음 문장을 위해 인덱스를 갱신

    return {
        "doc_key": doc["doc_key"],  # 문서 고유 ID
        "sentences": doc["sentences"],  # 문장의 원본 정보
        "clusters": doc["clusters"],  # coreference span 정보
        "tokens": tokens,  # 전체 문장을 하나로 이어붙인 토큰 리스트
        "sentence_spans": sentence_spans  # 각 문장이 문서 전체에서 차지하는 범위
    }
        
doc = litbank_jsons[0]
pre_doc = preprocess_litbank_doc(doc)
pre_doc.keys()

dict_keys(['doc_key', 'sentences', 'clusters', 'tokens', 'sentence_spans'])

In [106]:
print('clusters : ', pre_doc['clusters'])
print('clusters len : ', len(pre_doc['clusters']))

print('tokens : ', pre_doc['tokens'])
print('sentences : ', pre_doc['sentences'])
print('sentence_spans : ', pre_doc['sentence_spans'])
print('sentences len : ', len(pre_doc['sentences']))
print('sentences_spans len : ', len(pre_doc['sentence_spans']))

clusters :  [[[9, 10], [900, 900], [905, 905]], [[744, 744], [725, 733], [874, 875], [680, 681], [12, 13], [461, 462], [458, 459]], [[450, 454], [18, 19], [22, 23], [247, 250], [41, 41]], [[35, 36], [445, 447]], [[38, 39]], [[25, 39]], [[28, 39]], [[43, 53], [605, 606], [144, 145]], [[48, 53]], [[55, 56]], [[66, 87]], [[86, 87]], [[92, 93]], [[95, 97]], [[99, 101]], [[109, 110]], [[112, 114]], [[123, 124]], [[134, 135]], [[137, 138]], [[141, 142]], [[162, 163]], [[157, 168]], [[167, 168]], [[746, 746], [439, 439], [171, 174]], [[189, 189]], [[198, 199]], [[194, 199]], [[207, 207]], [[209, 209], [393, 393]], [[201, 209]], [[279, 279], [228, 229], [273, 277], [213, 217], [422, 426], [456, 456]], [[224, 225]], [[228, 232]], [[345, 346], [234, 235]], [[238, 242]], [[382, 382], [296, 296]], [[290, 298]], [[314, 314], [298, 298]], [[300, 301]], [[309, 310]], [[334, 335]], [[330, 335]], [[338, 339]], [[338, 341]], [[360, 360], [351, 351]], [[353, 357]], [[362, 366]], [[365, 366]], [[374, 374]

In [107]:
tokens = pre_doc["tokens"] # 전체 토큰 리스트
sentence_spans = pre_doc["sentence_spans"] # 문장 별 토큰 범위 리스트
num_tokens = len(tokens) # 전체 토큰 수
split_count = 2
split_points = [i * num_tokens // split_count for i in range(1, split_count)]
print('sentnece_spans : ', sentence_spans)
print('num_tokens : ', num_tokens)
print('split_points : ', split_points)

sentnece_spans :  [(0, 21), (22, 40), (41, 54), (55, 88), (89, 139), (140, 175), (176, 200), (201, 212), (213, 219), (220, 243), (244, 272), (273, 312), (313, 414), (415, 460), (461, 491), (492, 529), (530, 538), (539, 588), (589, 609), (610, 642), (643, 671), (672, 716), (717, 747), (748, 756), (757, 787), (788, 826), (827, 839), (840, 852), (853, 877), (878, 893), (894, 899), (900, 918), (919, 932), (933, 947), (948, 954), (955, 990), (991, 996), (997, 1015), (1016, 1027), (1028, 1033), (1034, 1055), (1056, 1065), (1066, 1075), (1076, 1089), (1090, 1107), (1108, 1114), (1115, 1126), (1127, 1136), (1137, 1145), (1146, 1153), (1154, 1160), (1161, 1176), (1177, 1190), (1191, 1207), (1208, 1223), (1224, 1242), (1243, 1263), (1264, 1273), (1274, 1279), (1280, 1290), (1291, 1326), (1327, 1345), (1346, 1364), (1365, 1379), (1380, 1383), (1384, 1401), (1402, 1418), (1419, 1424), (1425, 1432), (1433, 1486), (1487, 1519), (1520, 1534), (1535, 1546), (1547, 1563), (1564, 1583), (1584, 1597), (1

In [108]:
split_idxs = [] # 실제 문장을 자를 인덱스 리스트 (문장 단위로 나눌 것)
for sp in split_points: # 각 분할 기준에 대해서(마지막 문장 아니면 그냥 중간값인 split_points)
    # ex: sp = 1003
    for i, (start, end) in enumerate(sentence_spans): # 문장 범위를 돌면서 
        # ex: i = 0, (start,end) = (0,21) -> 첫 번째 문장의 길이가 22이므로 시작 인덱스는 0, 끝 인덱스는 21
        if end >= sp: # 끝 인덱스가 sp보다 크거나 같으면 해당 문장으로 자르기
            #  end = ? > sp = 1003 되는 순간
            print('끝 인덱스가 sp보다 커지는 시점 : ', sentence_spans[i]) 
            # 37번째 문장에서 끝 인덱스가 1003보다 크므로
            split_idxs.append(i + 1) # 그 다음 문장부터 자르기 위해 +1 
            # ex: split_idxs = [38] 
            break
    
print('split_idx : ', split_idxs)
# 구간 생성 : [처음 시작 문장 인덱스, 문장 자르는 문장 인덱스, 마지막 문장 인덱스]
split_idxs = [0] + split_idxs + [len(doc["sentences"])] # [0] + [38] + [106] -> [0, 38, 106] -> 0번째 문장부터 38번째 문장까지, 38번째 문장부터 106번째 문장까지
print('split_idxs : ', split_idxs)

끝 인덱스가 sp보다 커지는 시점 :  (997, 1015)
split_idx :  [38]
split_idxs :  [0, 38, 106]


In [109]:
# 분할 구간마다 새 문서 만들기
split_docs = []
for i in range(split_count): # 나눌 문서 수만큼 반복(2번, 마지막 문장만 3번)
    sent_start, sent_end = split_idxs[i], split_idxs[i + 1] # 문장 인덱스 범위 추출
    # 처음 반복 : 0, 38
    # 두 번째 반복 : 38, 106
    print('sent_start : ', sent_start, 'sent_end : ', sent_end)
    token_offset = sentence_spans[sent_start][0] if sent_start < len(sentence_spans) else 0 # 전체 문서에서 현재 부분 문서 시작하는 첫 토큰 위치
    # ex: token_offset = 0
    # ex: token_offset = 1016
    print('token_offset : ', token_offset)
    new_sentences = doc["sentences"][sent_start:sent_end] # 해당 범위의 문장들을 잘라냄
    print('new_sentences : ', new_sentences)
    new_token_count = sum(len(s) for s in new_sentences) # 해당 범위의 전체 토큰 수 계산
    print('new_token_count : ', new_token_count)
    # ex: 1번째 token 수=1016, 2번째 token 수=990 -> 1016 + 990 = 2006(전체 토큰 수)

    new_clusters = [] # 새 문서에서 사용할 클러스터 리스트
    for cluster in doc["clusters"]: # 원래 문서의 각 클러스터를 확인하면서
        new_cluster = [] # 클러스터 내에서 새로운 span을 저장
        #ex : cluster = [[[9, 10], [900, 900], [905, 905]], [[744, 744], [725, 733], [874, 875], [680, 681], [12, 13], [461, 462], [458, 459]]]
        for span in cluster: # 각 클러스터의 span
            #ex : span = [[9, 10], [900, 900], [905, 905]]
            if token_offset <= span[0] <= span[1] < token_offset + new_token_count: # 이 span이 현재 분할 범위 안에 **완전히** 포함된다면
                #ex : 0 <= [9,10] <= [900,900] < 0 + 1016 -> 성립함 -> new_cluster에 추가   
                #ex : 0 <= [1200,1200] <= [1300,1300] < 0 + 1016 -> 성립하지 않음 -> new_cluster에 추가안됨
                #ex : 1016 <= [1200,1200] <= [1300,1300] < 1016 + 990 -> 성립함 -> new_cluster에 추가
                new_cluster.append([span[0] - token_offset, span[1] - token_offset]) # 새로운 문서 기준으로 인덱스를 보정해서 저장
                #ex: append( [9,10] - 0, [900,900] - 0) -> [[9,10], [900,900]])
                #ex : append    ( [1200,1200] - 1016, [1300,1300] - 1016) -> [[184,184], [284,284]]
        if new_cluster:
            new_clusters.append(new_cluster) # 유효한 span만 모은 클러스터를 저장

    new_doc = {
        "doc_key": f"{doc['doc_key']}_{i}",
        "sentences": new_sentences,
        "clusters": new_clusters,
    }
    split_docs.append(new_doc)

sent_start :  0 sent_end :  38
token_offset :  0
new_sentences :  [['PART', 'ONE', 'CHAPTER', 'I', 'THE', 'EARLY', 'MARRIED', 'LIFE', 'OF', 'THE', 'MORELS', '“', 'THE', 'BOTTOMS', '”', 'succeeded', 'to', '“', 'Hell', 'Row', '”', '.'], ['Hell', 'Row', 'was', 'a', 'block', 'of', 'thatched', ',', 'bulging', 'cottages', 'that', 'stood', 'by', 'the', 'brookside', 'on', 'Greenhill', 'Lane', '.'], ['There', 'lived', 'the', 'colliers', 'who', 'worked', 'in', 'the', 'little', 'gin-pits', 'two', 'fields', 'away', '.'], ['The', 'brook', 'ran', 'under', 'the', 'alder', 'trees', ',', 'scarcely', 'soiled', 'by', 'these', 'small', 'mines', ',', 'whose', 'coal', 'was', 'drawn', 'to', 'the', 'surface', 'by', 'donkeys', 'that', 'plodded', 'wearily', 'in', 'a', 'circle', 'round', 'a', 'gin', '.'], ['And', 'all', 'over', 'the', 'countryside', 'were', 'these', 'same', 'pits', ',', 'some', 'of', 'which', 'had', 'been', 'worked', 'in', 'the', 'time', 'of', 'Charles', 'II', ',', 'the', 'few', 'colliers', 'and

### 5. 복원 확인(Json -> DataFrame)

In [149]:
def calculate_new_offsets(ontonotes_sentence, coreference_offsets):
    """
    주어진 coreference offset이 포함된 문장만 추출하여 순서대로 정리하고,
    문장이 일부만 추출되었기 때문에 offset 값을 새로운 기준에 맞게 조정한다.
    
    Args:
        ontonotes_sentence (list of lists): 전체 문장 리스트 (각 문장은 단어 리스트)
        coreference_offsets (list of lists): coreference 표현의 시작/끝 단어 인덱스 쌍

    Returns:
        tuple: 추출된 문장 리스트, 새롭게 조정된 offset 리스트
    """
    unique_sentences = []
    adjusted_offsets = []
    
    # 누적 길이를 추적하여 새 offset 계산에 활용
    cumulative_length = 0
    previous_length = 0

    for offset in coreference_offsets:
        for sentence_idx, sentence in enumerate(ontonotes_sentence):
            sentence_start = sum(len(s) for s in ontonotes_sentence[:sentence_idx])
            sentence_end = sentence_start + len(sentence) - 1

            # 현재 offset이 해당 문장 범위에 포함되는지 확인
            if sentence_start <= offset[0] <= sentence_end:
                if sentence not in unique_sentences:
                    unique_sentences.append(sentence)
                    cumulative_length += previous_length  # 앞선 문장 길이 누적
                # 새로운 offset 계산
                new_offset_start = offset[0] - sentence_start
                new_offset_end = new_offset_start + (offset[1] - offset[0])
                adjusted_offset = [
                    new_offset_start + cumulative_length,
                    new_offset_end + cumulative_length
                ]
                adjusted_offsets.append(adjusted_offset)
                previous_length = len(unique_sentences[-1])
                break

    return unique_sentences, adjusted_offsets

def find_coreference_terms(ontonotes_format, clusters_token_offsets):
    """
    주어진 offset으로부터 coreference cluster에 해당하는 실제 단어들을 추출한다.

    Args:
        ontonotes_format (list of list): 전체 문장 구조 (단어 리스트의 리스트)
        clusters_token_offsets (list of lists): 각 cluster에 속하는 단어 범위(offset)

    Returns:
        list: 각 cluster별로 단어 그룹을 저장한 리스트
    """
    result = []
    sentence_lengths = [len(sentence) for sentence in ontonotes_format]  # 문장별 길이

    for cluster in clusters_token_offsets:
        if len(cluster) == 1:
            continue  # 하나만 있으면 coref로 볼 수 없으므로 skip

        cluster_terms = []
        for start, end in cluster:
            sentence_idx = 0
            for i, length in enumerate(sentence_lengths):
                if start < length:
                    sentence_idx = i
                    break
                start -= length
                end -= length
            # 해당 문장에서 단어 추출
            terms = ontonotes_format[sentence_idx][start:end+1]
            cluster_terms.append(terms)
        result.append(cluster_terms)

    return result

def sort_by_first_value(data):
    """
    주어진 리스트를 각 원소의 첫 번째 값 기준으로 정렬한다.
    예: [[5, 6], [1, 2]] -> [[1, 2], [5, 6]]
    """
    return sorted(data, key=lambda x: x[0])

def checking(json_file):
    litbank_cases = []
    for index in range(len(json_file)):  # LitBank train 데이터셋에서 각 케이스(index)에 대해 반복
        coref_list = []
        litbank_case = pd.DataFrame()

        # 문자열 형태로 저장된 데이터를 실제 리스트로 변환
        for col in litbank_case.columns:
            litbank_case[col] = litbank_case[col].apply(ast.literal_eval)

        # Coreference cluster 추출
        for data in json_file['clusters'][index]:
            coref = data
            if len(coref) == 1:
                continue  # 하나만 있는 경우는 생략
            coref_list.append(coref)
            
        litbank_case['coref'] = coref_list

        # 문장과 offset 추출 및 재조정
        extracted_sentence_list, adjusted_offsets_list = [], []
        for data in litbank_case['coref']:
            extracted_sentence, adjusted_offsets = calculate_new_offsets(json_file['sentences'][index], data)
            extracted_sentence_list.append(extracted_sentence)
            adjusted_offsets_list.append(adjusted_offsets)
            
        litbank_case['extracted_sentence'] = extracted_sentence_list
        litbank_case['adjusted_offsets'] = adjusted_offsets_list

        # offset 정렬
        litbank_case['adjusted_offsets'] = litbank_case['adjusted_offsets'].apply(lambda x: sort_by_first_value(x))

        # 실제 단어 표현 추출
        litbank_case['text'] = find_coreference_terms(json_file['sentences'][index], litbank_case['coref'])

        litbank_cases.append(litbank_case)
    return litbank_cases

In [187]:
litbank_jsons=[]
with jsonlines.open("data/LitBank/train.english.jsonlines") as read_file:
    for line in read_file.iter():
        litbank_jsons.append(line)

split_litbank_jsons=[]
with jsonlines.open("data/LitBank/train-splitted.english.jsonlines") as read_file:
    for line in read_file.iter():
        split_litbank_jsons.append(line)
        
new_split_litbank_jsons=[]
with jsonlines.open("data/new_train-splitted.english.jsonlines") as read_file:
    for line in read_file.iter():
        new_split_litbank_jsons.append(line)

old_augment_litbank_jsons = []
with jsonlines.open("data/litbank_augment.jsonlines") as read_file:
    for line in read_file.iter():
        old_augment_litbank_jsons.append(line)
        
litbank_iaa_jsons=[]
with jsonlines.open("data/litbank_augment_iaa.jsonlines") as read_file:
    for line in read_file.iter():
        litbank_iaa_jsons.append(line)

split_litbank_iaa_jsons=[]
with jsonlines.open("data/splitted_litbank_augment_iaa.jsonlines") as read_file:
    for line in read_file.iter():
        split_litbank_iaa_jsons.append(line)
    

In [204]:
# 원본 litbank 데이터
litbank_json = pd.DataFrame(litbank_jsons)[['doc_key', 'sentences', 'clusters']]
litbank_dfs = checking(litbank_json)
litbank_df = litbank_dfs[0]
litbank_df.head(5)

,coref,extracted_sentence,adjusted_offsets,text
0,"[[9, 10], [900, 900], [905, 905]]","[[PART, ONE, CHAPTER, I, THE, EARLY, MARRIED, ...","[[9, 10], [22, 22], [27, 27]]","[[THE, MORELS], [They], [their]]"
1,"[[744, 744], [725, 733], [874, 875], [680, 681...","[[Mrs., Morel, was, not, anxious, to, move, in...","[[8, 16], [27, 27], [52, 53], [64, 65], [113, ...","[[it], [the, Bottoms, ,, which, was, already, ..."
2,"[[450, 454], [18, 19], [22, 23], [247, 250], [...","[[To, accommodate, the, regiments, of, miners,...","[[35, 39], [64, 65], [68, 69], [90, 93], [116,...","[[the, site, of, Hell, Row], [Hell, Row], [Hel..."
3,"[[35, 36], [445, 447]]","[[Hell, Row, was, a, block, of, thatched, ,, b...","[[13, 14], [49, 51]]","[[the, brookside], [the, brook, valley]]"
4,"[[43, 53], [605, 606], [144, 145]]","[[There, lived, the, colliers, who, worked, in...","[[2, 12], [30, 31], [39, 40]]","[[the, colliers, who, worked, in, the, little,..."


In [205]:
# 원본 split litbank 데이터
split_litbank_json = pd.DataFrame(split_litbank_jsons)[['doc_key', 'sentences', 'clusters']]
split_litbank_dfs = checking(split_litbank_json)
split_litbank_df_0 = split_litbank_dfs[0]
split_litbank_df_0.head(5)

,coref,extracted_sentence,adjusted_offsets,text
0,"[[9, 10], [900, 900], [905, 905]]","[[PART, ONE, CHAPTER, I, THE, EARLY, MARRIED, ...","[[9, 10], [22, 22], [27, 27]]","[[THE, MORELS], [They], [their]]"
1,"[[744, 744], [725, 733], [874, 875], [680, 681...","[[Mrs., Morel, was, not, anxious, to, move, in...","[[8, 16], [27, 27], [52, 53], [64, 65], [113, ...","[[it], [the, Bottoms, ,, which, was, already, ..."
2,"[[450, 454], [18, 19], [22, 23], [247, 250], [...","[[To, accommodate, the, regiments, of, miners,...","[[35, 39], [64, 65], [68, 69], [90, 93], [116,...","[[the, site, of, Hell, Row], [Hell, Row], [Hel..."
3,"[[35, 36], [445, 447]]","[[Hell, Row, was, a, block, of, thatched, ,, b...","[[13, 14], [49, 51]]","[[the, brookside], [the, brook, valley]]"
4,"[[43, 53], [605, 606], [144, 145]]","[[There, lived, the, colliers, who, worked, in...","[[2, 12], [30, 31], [39, 40]]","[[the, colliers, who, worked, in, the, little,..."


In [206]:
# 새로운 split litbank 데이터
new_split_litbank_json = pd.DataFrame(new_split_litbank_jsons)[['doc_key', 'sentences', 'clusters']]
new_split_litbank_dfs = checking(new_split_litbank_json)
new_split_litbank_df_0 = new_split_litbank_dfs[0]
new_split_litbank_df_0.head(5)

,coref,extracted_sentence,adjusted_offsets,text
0,"[[9, 10], [900, 900], [905, 905]]","[[PART, ONE, CHAPTER, I, THE, EARLY, MARRIED, ...","[[9, 10], [22, 22], [27, 27]]","[[THE, MORELS], [They], [their]]"
1,"[[744, 744], [725, 733], [874, 875], [680, 681...","[[Mrs., Morel, was, not, anxious, to, move, in...","[[8, 16], [27, 27], [52, 53], [64, 65], [113, ...","[[it], [the, Bottoms, ,, which, was, already, ..."
2,"[[450, 454], [18, 19], [22, 23], [247, 250], [...","[[To, accommodate, the, regiments, of, miners,...","[[35, 39], [64, 65], [68, 69], [90, 93], [116,...","[[the, site, of, Hell, Row], [Hell, Row], [Hel..."
3,"[[35, 36], [445, 447]]","[[Hell, Row, was, a, block, of, thatched, ,, b...","[[13, 14], [49, 51]]","[[the, brookside], [the, brook, valley]]"
4,"[[43, 53], [605, 606], [144, 145]]","[[There, lived, the, colliers, who, worked, in...","[[2, 12], [30, 31], [39, 40]]","[[the, colliers, who, worked, in, the, little,..."


In [191]:
# 원본 split litbank 데이터
split_litbank_json = pd.DataFrame(split_litbank_jsons)[['doc_key', 'sentences', 'clusters']]
split_litbank_dfs = checking(split_litbank_json)
split_litbank_df_1= split_litbank_dfs[1]
split_litbank_df_1

,coref,extracted_sentence,adjusted_offsets,text
0,"[[329, 329], [458, 458], [357, 357], [711, 711...","[[She, left, him, alone, ,, and, went, to, get...","[[0, 0], [21, 21], [24, 24], [41, 41], [88, 88...","[[She], [her], [You], [she], [her], [her], [sh..."
1,"[[739, 740], [799, 800]]","[[As, she, crossed, the, open, ground, in, fro...","[[29, 30], [39, 40]]","[[her, husband], [my, dad]]"
2,"[[384, 384], [105, 105], [345, 346], [532, 532...","[[--, that, lion, 's, killed, three, men, --, ...","[[11, 11], [25, 25], [57, 58], [87, 87], [96, ...","[[my], [your], [the, lad], [he], [his], [He], ..."
3,"[[707, 708], [239, 240]]","[[And, she, went, slowly, away, with, her, lit...","[[31, 32], [39, 40]]","[[the, wakes], [the, wakes]]"
4,"[[338, 338], [156, 156], [178, 178], [208, 209...","[[She, left, him, alone, ,, and, went, to, get...","[[9, 9], [16, 16], [38, 38], [55, 55], [57, 61...","[[Annie], [I], [you], [her, child], [Annie], [..."
5,"[[33, 33], [28, 29]]","[[They, were, eating, batter-pudding, and, jam...","[[0, 0], [30, 31]]","[[They], [the, three]]"


In [207]:
# 새로운 split litbank 데이터
new_split_litbank_json = pd.DataFrame(new_split_litbank_jsons)[['doc_key', 'sentences', 'clusters']]
new_split_litbank_dfs = checking(new_split_litbank_json)
new_split_litbank_df_1= new_split_litbank_dfs[1]
new_split_litbank_df_1

,coref,extracted_sentence,adjusted_offsets,text
0,"[[504, 504], [633, 633], [89, 90], [532, 532],...","[[She, left, him, alone, ,, and, went, to, get...","[[0, 0], [21, 21], [38, 39], [42, 42], [59, 59...","[[She], [her], [the, mother], [You], [she], [h..."
1,"[[914, 915], [974, 975]]","[[As, she, crossed, the, open, ground, in, fro...","[[29, 30], [39, 40]]","[[her, husband], [my, dad]]"
2,"[[559, 559], [280, 280], [520, 521], [707, 707...","[[--, that, lion, 's, killed, three, men, --, ...","[[11, 11], [25, 25], [57, 58], [87, 87], [96, ...","[[my], [your], [the, lad], [he], [his], [He], ..."
3,"[[882, 883], [7, 8], [414, 415]]","[[And, she, went, slowly, away, with, her, lit...","[[31, 32], [41, 42], [51, 52]]","[[the, wakes], [the, wakes], [the, wakes]]"
4,"[[513, 513], [331, 331], [353, 353], [383, 384...","[[She, left, him, alone, ,, and, went, to, get...","[[9, 9], [16, 16], [38, 38], [55, 55], [62, 62...","[[Annie], [I], [you], [her, child], [Annie], [..."
5,"[[146, 146], [167, 167]]","[[“, They, 'll, be, beginnin, ', ,, ”, the, bo...","[[1, 1], [22, 22]]","[[They], [they]]"
6,"[[208, 208], [203, 204]]","[[They, were, eating, batter-pudding, and, jam...","[[0, 0], [30, 31]]","[[They], [the, three]]"


In [210]:
# litbank iaa 데이터
litbank_iaa_json = pd.DataFrame(litbank_iaa_jsons)[['doc_key', 'sentences', 'clusters']]
litbank_iaa_dfs = checking(litbank_iaa_json)
litbank_iaa_df = litbank_iaa_dfs[0]
litbank_iaa_df.head(5)

,coref,extracted_sentence,adjusted_offsets,text
0,"[[9, 10], [902, 902], [907, 907]]","[[PART, ONE, CHAPTER, I, THE, EARLY, MARRIED, ...","[[9, 10], [27, 27], [32, 32]]","[[THE, MORELS], [.], [in]]"
1,"[[12, 13], [459, 460], [462, 463], [682, 683],...","[[PART, ONE, CHAPTER, I, THE, EARLY, MARRIED, ...","[[12, 13], [64, 65], [67, 68], [75, 76], [120,...","[[THE, BOTTOMS], [erected, the], [.], [in, the..."
2,"[[18, 19], [22, 23], [41, 41], [248, 251], [45...","[[PART, ONE, CHAPTER, I, THE, EARLY, MARRIED, ...","[[18, 19], [22, 23], [41, 41], [58, 61], [119,...","[[Hell, Row], [Hell, Row], [There], [time, the..."
3,"[[35, 36], [446, 448]]","[[Hell, Row, was, a, block, of, thatched, ,, b...","[[13, 14], [48, 50]]","[[the, brookside], [in, the, brook]]"
4,"[[43, 53], [144, 146], [608, 610]]","[[There, lived, the, hardworking, colliers, wh...","[[2, 12], [18, 20], [69, 71]]","[[the, hardworking, colliers, who, worked, in,..."


In [209]:
# old augment litbank 데이터
old_augment_litbank_json = pd.DataFrame(old_augment_litbank_jsons)[['doc_key', 'sentences', 'clusters']]
old_augment_litbank_dfs = checking(old_augment_litbank_json)
old_augment_litbank_df = old_augment_litbank_dfs[0]
old_augment_litbank_df.head(5)

,coref,extracted_sentence,adjusted_offsets,text
0,"[[9, 10], [902, 902], [907, 907]]","[[PART, ONE, CHAPTER, I, THE, EARLY, MARRIED, ...","[[9, 10], [23, 23], [28, 28]]","[[THE, MORELS], [They], [their]]"
1,"[[746, 746], [727, 735], [876, 877], [682, 683...","[[Mrs., Morel, was, not, anxious, to, move, in...","[[8, 16], [27, 27], [52, 53], [64, 65], [113, ...","[[it], [the, Bottoms, ,, which, was, already, ..."
2,"[[452, 456], [18, 20], [23, 25], [249, 252], [...","[[To, accommodate, the, regiments, of, miners,...","[[35, 39], [64, 66], [69, 71], [92, 95], [118,...","[[the, site, of, Hell, Row], [notorious, Hell,..."
3,"[[35, 36], [445, 447]]","[[notorious, Hell, Row, was, a, block, of, tha...","[[12, 13], [48, 50]]","[[stood, by], [,, in, the]]"
4,"[[43, 53], [607, 608], [144, 145]]","[[There, lived, the, colliers, who, worked, in...","[[0, 10], [30, 31], [37, 38]]","[[There, lived, the, colliers, who, worked, in..."
